In [83]:
# import
import pandas as pd
import numpy as np

from colorama import Fore, Back, Style

In [ ]:
# load cleaned data
dataset_lowercase = pd.read_csv('Cleaned_Book_ETL.csv', encoding='UTF-8', sep=',')
dataset_lowercase.head()


In [44]:
# zůžení výběru na TOLKIEN readery a knihu
book_condtion = dataset_lowercase['Book-Title']=='the fellowship of the ring (the lord of the rings, part 1)'
autor_condition = dataset_lowercase['Book-Author'].str.contains("tolkien")

# readers Seires
tolkien_readers = dataset_lowercase.loc[((book_condtion) & autor_condition), 'User-ID']

In [ ]:
# to_list replace with list() A) READERS
tolkien_readers = list(tolkien_readers)
tolkien_readers = np.unique(tolkien_readers)
tolkien_readers # ndarray

In [ ]:
# 'final' dataset to test a recommendation
books_of_tolkien_readers = dataset_lowercase[(dataset_lowercase['User-ID'].isin(tolkien_readers))]
books_of_tolkien_readers.head()

In [ ]:
# Number of ratings per other books in dataset
number_of_rating_per_book = books_of_tolkien_readers.groupby('Book-Title').agg('count').reset_index()

# check
number_of_rating_per_book.sort_values('User-ID', ascending=False) # AVG rating per user 1.28

,Book-Title,User-ID,ISBN,Book-Rating,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
13439,the fellowship of the ring (the lord of the ri...,205,205,205,205,205,205,205,205,205
15614,"the two towers (the lord of the rings, part 2)",75,75,75,75,75,75,75,75,75
14993,"the return of the king (the lord of the rings,...",73,73,73,73,73,73,73,73,73
13811,the hobbit : the enchanting prelude to the lor...,38,38,38,38,38,38,38,38,38
5768,harry potter and the chamber of secrets (book 2),37,37,37,37,37,37,37,37,37
...,...,...,...,...,...,...,...,...,...,...
6314,hybrids (neanderthal parallax),1,1,1,1,1,1,1,1,1
6316,hyperthought,1,1,1,1,1,1,1,1,1
6317,"hгѓ?г‚в¶re, gгѓ?г‚в¶ttliche freundin. gebete u...",1,1,1,1,1,1,1,1,1
6318,hгѓ?г‚вјter der pforten. geschichten aus dem c...,1,1,1,1,1,1,1,1,1


In [72]:
#select only books which have actually higher number of ratings than threshold - arbitrary decision, but relevant

books_to_compare = number_of_rating_per_book.loc[number_of_rating_per_book['Book-Rating'] > 8, 'Book-Title'] # arbitrary threshold decision

# to_list replace with list() B) BOOKS
books_to_compare = list(books_to_compare) # list of the higher rated books than 8 count of ratings
books_to_compare

['1984',
 '1st to die: a novel',
 'a painted house',
 'a time to kill',
 'a wrinkle in time',
 'angels &amp; demons',
 "bridget jones's diary",
 'dreamcatcher',
 "ender's game (ender wiggins saga (paperback))",
 'fahrenheit 451',
 'firestarter',
 'hannibal',
 'harry potter and the chamber of secrets (book 2)',
 'harry potter and the goblet of fire (book 4)',
 'harry potter and the order of the phoenix (book 5)',
 'harry potter and the prisoner of azkaban (book 3)',
 "harry potter and the sorcerer's stone (book 1)",
 "harry potter and the sorcerer's stone (harry potter (paperback))",
 'house of sand and fog',
 'interview with the vampire',
 'it',
 'jurassic park',
 'life of pi',
 'lightning',
 'pet sematary',
 'red dragon',
 'stardust',
 'the bad beginning (a series of unfortunate events, book 1)',
 'the brethren',
 'the client',
 'the da vinci code',
 'the drawing of the three (the dark tower, book 2)',
 'the fellowship of the ring (the lord of the rings, part 1)',
 'the firm',
 'the g

In [75]:
ratings_data_raw = books_of_tolkien_readers[['User-ID', 'Book-Title', 'Book-Rating']].loc[books_of_tolkien_readers['Book-Title'].isin(books_to_compare)] # list of the higher rated books than 8 count of ratings
ratings_data_raw.sort_values(by='Book-Rating', ascending=False)

,User-ID,Book-Title,Book-Rating
205809,142149,the fellowship of the ring (the lord of the ri...,10
127306,88733,"the two towers (the lord of the rings, part 2)",10
153219,102702,harry potter and the sorcerer's stone (harry p...,10
255279,179734,harry potter and the goblet of fire (book 4),10
153164,102702,harry potter and the goblet of fire (book 4),10
...,...,...,...
314592,225379,the lovely bones: a novel,2
163212,110265,the fellowship of the ring (the lord of the ri...,2
18356,11676,pet sematary,2
232232,162738,hannibal,1


In [79]:
# Check the duplicates
# ratings_data_raw_with_duplicates = ratings_data_raw.copy()
# Check ratings_data_raw_with_duplicates[ratings_data_raw_with_duplicates.duplicated(subset=['User-ID', 'Book-Title'])].sort_values(by=['User-ID', 'Book-Title'], ascending=False)

# Drop duplicates - 58 duplicates = users rated the same book at least twice

In [ ]:
# group by User and Book and compute mean
ratings_data_raw_nodup = ratings_data_raw.groupby(['User-ID', 'Book-Title']).agg('Book-Rating').mean().reset_index()
ratings_data_raw_nodup[ratings_data_raw_nodup.duplicated(subset=['User-ID', 'Book-Title'])] # no duplicates

ratings_data_raw_nodup

In [ ]:
# pivot table
dataset_for_corr = ratings_data_raw_nodup.pivot(index='User-ID', columns='Book-Title', values='Book-Rating')
dataset_for_corr

In [82]:
LoR_list = ['the fellowship of the ring (the lord of the rings, part 1)']

result_list = []
worst_list = []

# for each of the trilogy book compute:
for LoR_book in LoR_list:
    
    #Take out the Lord of the Rings selected book from correlation dataframe
    dataset_of_other_books = dataset_for_corr.copy(deep=False) # soft copy (ale při úpravách (například odstranění sloupce) se změny neprojeví v původním datovém rámci)
    dataset_of_other_books.drop([LoR_book], axis=1, inplace=True) # remove the selected book from the correlation dataframe # nezasahuje do jednotlivých hodnot datového rámce, ale pouze manipuluje s jeho strukturou.
      
    # empty lists
    book_titles = []
    correlations = []
    avgrating = []

    # corr computation
    for book_title in list(dataset_of_other_books.columns.values): # pro každou knihu v datasetu hledáme korelaci s vybranou knihou jinou než LoR
        book_titles.append(book_title)
        correlations.append(dataset_for_corr[LoR_book].corr(method='spearman', other=dataset_of_other_books[book_title])) # Pearson correlation - Korelace v tomto případě ukazuje, zda tento konkrétní uživatel hodnotí dvě knihy ve vztahu k průměrnému hodnocení těchto knih podobným způsobem jako ostatní uživatelé
        tab=(ratings_data_raw[ratings_data_raw['Book-Title']==book_title])
        avgrating.append(tab['Book-Rating'].min())
    
    # final dataframe of all correlation of each book   
    corr_fellowship = pd.DataFrame(list(zip(book_titles, correlations, avgrating)), columns=['book','corr','avg_rating'])

    # top 10 books with highest corr
    result_list.append(corr_fellowship.sort_values('corr', ascending = False))
        
        #worst 10 books
    worst_list.append(corr_fellowship.sort_values('corr', ascending = False).tail(10))

# result_list[0][result_list[0]['book'] == 'the two towers (the lord of the rings, part 2)']
result_list[0].head(10)


,book,corr,avg_rating
42,"the return of the king (the lord of the rings,...",0.804676,5
26,stardust,0.760767,5
29,the client,0.760377,5
54,watership down,0.757189,5
8,ender's game (ender wiggins saga (paperback)),0.753778,8
44,the silmarillion,0.733246,5
20,it,0.718049,5
7,dreamcatcher,0.706151,3
24,pet sematary,0.694564,2
4,a wrinkle in time,0.687413,5


In [ ]:
ratings_data_raw_nodup[ratings_data_raw_nodup['Book-Title'].isin(['the fellowship of the ring (the lord of the rings, part 1)', 'the return of the king (the lord of the rings, part 3)'])].sort_values(by='User-ID', ascending=False)

# Vybereme pouze řádky, kde je titul jedním z obou knih
filtered_df = ratings_data_raw_nodup[ratings_data_raw_nodup['Book-Title'].isin([
    'the fellowship of the ring (the lord of the rings, part 1)',
    'the return of the king (the lord of the rings, part 3)'
])]

# Skupinujeme podle 'User-ID' a počítáme počet různých titulů hodnocených každým uživatelem
user_counts = filtered_df.groupby('User-ID')['Book-Title'].nunique()
user_indexes = user_counts[user_counts == 2]

ratings_data_raw_nodup[ratings_data_raw_nodup['User-ID'].isin(user_indexes.index)]


In [ ]:

# print("Correlation for book:", LoR_list[0])
print("Average rating of LOR:", ratings_data_raw.loc[ratings_data_raw['Book-Title']=='the fellowship of the ring (the lord of the rings, part 1)'].groupby('Book-Title').agg('Book-Rating').mean())

rslt = len(result_list)
rslt

In [88]:
# Např. vážený průměr = (průměrné hodnocení × počet hodnocení) / celkový počet hodnocení.

# Např. vybrat pouze knihy, které mají alespoň 5 hodnocení, abyste eliminovali tituly s jedním uživatelem, který dal extrémně vysokou nebo nízkou známku.